In [1]:
!pip install allennlp==2.1.0 allennlp-models==2.1.0
import spacy
nlp = spacy.load('en_core_web_sm')
import allennlp
import allennlp_models
from allennlp.models.archival import load_archive
from allennlp.common.util import JsonDict
from allennlp.data import Instance
from allennlp.predictors.predictor import Predictor
from allennlp.data.fields import LabelField
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer
from typing import List, Dict
from overrides import overrides

     |████████████████████████████████| 593kB 7.6MB/s 
     |████████████████████████████████| 409kB 13.4MB/s 
     |████████████████████████████████| 133kB 16.2MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 1.2MB 43.7MB/s 
     |████████████████████████████████| 12.8MB 23.8MB/s 
     |████████████████████████████████| 1.9MB 31.3MB/s 
     |████████████████████████████████| 122kB 50.0MB/s 
     |████████████████████████████████| 266kB 26.8MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 7.4MB 31.2MB/s 
     |████████████████████████████████| 3.3MB 48.0MB/s 
     |████████████████████████████████| 901kB 52.0MB/s 
  Created wheel for overrides: filename=overrides-3.1.0-cp37-none-any.whl size=10174 sha256=c8cb5c5a6fb7a39e431c311b9e132cbf7b2b869c57ec37e66fba56a986d84ab9
  Stored

In [2]:
!pip install lime
!pip install transformers
!pip install pytreebank
!pip install datasets==1.5.0

import os
import random

import pandas as pd
import numpy as np
import scipy as sp
import torch
import spacy
from torch.utils.data import \
    TensorDataset, \
    DataLoader
from transformers import \
    BertTokenizer, \
    BertForSequenceClassification, \
    AdamW, \
    BertConfig, \
    get_linear_schedule_with_warmup


     |████████████████████████████████| 276kB 8.6MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=c2947abc12e65d95a32ed08bb9bb0683e2b59eb386f2d213f27ace6eb3f5db12
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime
  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=12607b41423c1239741c8b1537429bc14fc705d5ac38e14ca06df09366c89571
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank
     |████████████████████████████████| 194kB 8.2MB/s 
     |████████████████████████████████| 112kB 14.5MB/s 
     |████████████████████████████████| 245kB 16.9MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
cd '/content/drive/MyDrive/NLP Project'

/content/drive/MyDrive/NLP Project


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# BERT Example

**Load your data**

In [7]:
import src.data.dataload
sst=src.data.dataload.load_sst()
train, val, test = sst.train_val_test

In [8]:
ag_news=src.data.dataload.load_agnews()
train_ag, _, test_ag = ag_news.train_val_test

Using custom data configuration default


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/17ec33e23df9e89565131f989e0fdf78b0cc4672337b582da83fc3c9f79fe34d. Subsequent calls will reuse this data.


**Set up your model & tokenizer**

In [9]:

bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP Project/BERT Model')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

id2label = bert_model.config.id2label
label2id = bert_model.config.label2id
labels = sorted(label2id, key=label2id.get)

**Explanations on SST - SHAP**

In [10]:
!pip install shap==0.39.0

     |████████████████████████████████| 358kB 7.5MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491621 sha256=0e00e24201976bc641ce10956aa545f824aebded17650b5371c6a992b3355c03
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [30]:
import importlib #this was needed only because I was modifying the modules
import explainers_functions #this was needed only because I was modifying the modules
importlib.reload(explainers_functions) #this was needed only because I was modifying the modules
from explainers_functions import *

In [ ]:
import shap

SHAPExplainer = SHAPExplainer(bert_model,tokenizer,labels,device)
tokens,values = SHAPExplainer.explain_instances(train['sentence'][0:10])

tokens,values

**Explanations on AG News**

In [13]:
'''
NOTE - the BERT model hasn't been fine-tuned on AG News so this is just to verify the mechanics on a different dataset
'''
import shap
SHAPExplainer = SHAPExplainer(bert_model,tokenizer,labels,device)
tokens,values = SHAPExplainer.explain_instances(train_ag['sentence'][0:5])

tokens,values

Partition explainer:  20%|██        | 1/5 [00:00<?, ?it/s]

Partition explainer:  60%|██████    | 3/5 [00:27<00:12,  6.47s/it]

Partition explainer:  80%|████████  | 4/5 [00:40<00:08,  8.43s/it]

Partition explainer: 100%|██████████| 5/5 [00:53<00:00,  9.81s/it]

Partition explainer: 6it [01:06, 11.07s/it]

(array([list([' ', 'reuters ', '- ', 'short ', '- ', 'sellers ', ', ', 'wall ', 'street ', "' ", 's ', 'd ', '##wind ', '##ling ', '##band ', 'of ', 'ultra ', '- ', 'cy ', '##nic ', '##s ', ', ', 'are ', 'seeing ', 'green ', 'again ', '. ', ' ']),
        list([' ', 'reuters ', '- ', 'private ', 'investment ', 'firm ', 'carly ', '##le ', 'group ', ', ', 'which ', 'has ', 'a ', 'reputation ', 'for ', 'making ', 'well ', '- ', 'timed ', 'and ', 'occasionally ', '##con ', '##tro ', '##vers ', '##ial ', 'plays ', 'in ', 'the ', 'defense ', 'industry ', ', ', 'has ', 'quietly ', 'placed ', '##its ', 'bets ', 'on ', 'another ', 'part ', 'of ', 'the ', 'market ', '. ', ' ']),
        list([' ', 'reuters ', '- ', 'soaring ', 'crude ', 'prices ', 'plus ', 'worries ', '##ab ', '##out ', 'the ', 'economy ', 'and ', 'the ', 'outlook ', 'for ', 'earnings ', 'are ', 'expected ', 'to ', '##hang ', 'over ', 'the ', 'stock ', 'market ', 'next ', 'week ', 'during ', 'the ', 'depth ', 'of ', 'the ', '##s

**Explanations on SST - LIME**

In [ ]:
LimeExplainer = LimeExplainer(bert_model,tokenizer,labels,device,'BERT')
tokens,values = LimeExplainer.explain_instances(train['sentence'][0:10])

tokens,values

# BCN Example

**Set up your model & tokenizer**

In [19]:
cd /content/drive/MyDrive/NLP Project/AllenNLP

/content/drive/MyDrive/NLP Project/AllenNLP


In [20]:
# importing the dataset reader
import tagging
# importing the BCN model
import BCN_model
archive = load_archive("./BCN_output/model.tar.gz")
BCN_model = archive.model
vocab = BCN_model.vocab
BCN_predictor = Predictor.from_archive(archive, 'ag_text_classifier')

**Explanations on LIME**

In [16]:
cd '/content/drive/MyDrive/NLP Project'

/content/drive/MyDrive/NLP Project


**Explanations on AG News**

In [31]:
labels_BCN = ['Sci/Tech', 'Sports','World','Business']
LimeExplainer = LimeExplainer(BCN_predictor,None,labels_BCN,device,'BCN')
indices=np.random.choice(len(test_ag), 20, replace=False)
instance_array = test_ag['sentence'].iloc[indices]
top_tokens_AG,top_values_AG=LimeExplainer.explain_instances(instance_array)

**Helper functions (model dependent)**

In [15]:
import numpy as np

@Predictor.register('ag_text_classifier')
class AGNewsClassifier(Predictor):
    """
    Predictor for any model that takes in a sentence and returns
    a single class for it.  In particular, it can be used with
    the [`BasicClassifier`](../models/basic_classifier.md) model.

    """

    def predict(self, sentence: str) -> JsonDict:
        return self.predict_json({"Description": sentence})

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        """
        Expects JSON that looks like `{"sentence": "..."}`.
        Runs the underlying model, and adds the `"label"` to the output.
        """
        sentence = json_dict["Description"]
        reader_has_tokenizer = (
            getattr(self._dataset_reader, "tokenizer", None) is not None
            or getattr(self._dataset_reader, "_tokenizer", None) is not None
        )
        if not reader_has_tokenizer:
            tokenizer = SpacyTokenizer()
            sentence = tokenizer.tokenize(sentence)
        return self._dataset_reader.text_to_instance(sentence)

    @overrides
    def predictions_to_labeled_instances(
        self, instance: Instance, outputs: Dict[str, np.ndarray]
    ) -> List[Instance]:
        new_instance = instance.duplicate()
        label = np.argmax(outputs["class_probabilities"])
        new_instance.add_field("label", LabelField(int(label), skip_indexing=True))
        return [new_instance]